# Version 0.2

In [1]:
# Imports

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [2]:
# Initialize LLM 

#llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
llm = OpenAI(temperature=0.0)

## (1) Speech Act Classification

In [3]:
## (1) Speech Act Classification 

template_speech_act_classifier= """
Decide whether the utterance below from a speaker to a listener is one of INSTRUCT, STATEMENT, GREETING, QUESTIONWH ('wh', questions), QUESTIONYN ('yes/no' questions), ACK (e.g. "yes" or "ok"), or UNKNOWN 
An INSTRUCT is an imperative statement or a request by the speaker to have the listener do an action or stop doing an action.
A QUESTIONWH is a 'wh' query (what, why, when, where, who) or request from a speaker for more information from the listener about the listeners knowledge, beliefs or perceptions
A QUESTIONYN is a 'yes/no' query or request from a speaker for more information from the listener about the listeners knowledge, beliefs or perceptions, but the speaker expects a yes or no for an answer
A STATEMENT is a statement of fact or opinion that the speaker conveys to a listener. 
A GREETING is an expression of social connection establishing the start of the conversation. E.g., "Hello"
A ACK is an acknowledgement (either "yes" or "no").
A UNKNOWN is an utterance not one of the above. 

utterance: \n{utterance}\n
act:
"""

prompt_speech_act_classifier = PromptTemplate(
    input_variables=["utterance"],
    template=template_speech_act_classifier
)

chain_speech_act_classifier = LLMChain(llm=llm, prompt=prompt_speech_act_classifier)


## (2) Central Referrent being talked about

In [4]:
"""
1. CENTRAL REF
What is the central item (which could be a single thing or a collection of things) that is being referred to in the below sentence?
sentence: stack some lemons on the table central 
referent: lemons
→ What are they talking about? 

"""

template_centralref = """
What is the central item (which could be a single thing or a collection of things) that is being referred to in the below sentence?

sentence: \n{utterance}\n 
referent:
"""

prompt_centralref = PromptTemplate(
    input_variables=["utterance"],
    template=template_centralref
)

chain_centralref = LLMChain(llm=llm, prompt=prompt_centralref)


## (3A) What central action is performed on central referent (if INSTRUCT)

In [5]:
"""
2a. ACTION ON CENTRAL REF (if INSTRUCT)
In the below sentence, describe the action that is being performed on the central referent 
sentence: stack some lemons on the table 
central referent: lemons |
action: stacking
→ What do they want me to do with the thing they are talking about? 

"""

template_centralaction = """
In the below sentence, describe the action that is being performed on the central referent 

sentence: \n{utterance}\n
central referent: \n{centralref}\n
action:
"""

prompt_centralaction = PromptTemplate(
    input_variables=["utterance", "centralref"],
    template=template_centralaction
)

chain_centralaction = LLMChain(llm=llm, prompt=prompt_centralaction)


## (4A) What "feasible action" (if any) is being described (if INSTRUCT)

In [6]:
"""
3a. ACTION GROUNDING (INSTRUCT)
3a-i. Generate 
Given a set of actions….select one that is close in meaning to "action". If none, then return NONE. If several, then return AMBIGUOUS.  
3a-ii. Test 
Check if the parameter types of the action sufficiently captures the nature of the referent. 
>> Iterate on 3a until stopping condition or action found. 

→ Can I really do the thing they want me to do? 
"""


# GENERATE [Select a candidate action]
template_candidateaction = """
Select an action from the list of available actions that is most relevant to the given central action performed on the central referent as understood in the context of the utterance. 
To decide the applicable action, use the following procedure to systematically filter the most relevant action:
1. Compare the name and description of the action to the central action. Narrow the list of actions to include only those with a semantically similar name or description to the central action. 
2. If the narrowed list contains one action, then return its name. If it contains no actions, then return NONE. If it contains more than one action, then return AMBIGUOUS.

\n\n LIST OF AVAILABLE ACTIONS \n:
{actions}

utterance: \n{utterance}\n
central referent: \n{centralref}\n
central action: \n{centralaction}\n
candidate action:
"""

prompt_candidateaction = PromptTemplate(
    input_variables=["utterance", "actions", "centralref", "centralaction"],
    template=template_candidateaction
)

chain_candidateaction = LLMChain(llm=llm, prompt=prompt_candidateaction)

# TEST
template_typeof = """
Determine whether or not the central referent item mentioned below is one of the types also provided below. To check if the referent is of a type, follow the below procedure
1. Iterate through each item mentioned in the list of types. 
2. For each item X in the list of types expand on the meaning of each item, and then ask if the central referent is of type X given that meaning. 
3. If the central referent is of type X in the list, return X.

\n\n EXAMPLE \n
central referent: lemon
types: ['area', 'physobj', 'location']
typeOf: Looking through the items in the list of types above. physobj is a physical object. lemon is a type of physical object. So, it is of type physobj

Remember, return specifically ONE of the items in the list, or if none apply then return NONE. 

central referent: \n{centralref}\n
types: \n{types}\n
typeOf:
"""

prompt_typeof = PromptTemplate(
    input_variables=["centralref", "types"],
    template=template_typeof
)

chain_typeof = LLMChain(llm=llm, prompt=prompt_typeof)

# Note: will need to input "types" which we can get from the actions list. Also, will need to post process as we use CoT
# Note: we also don't consider multiple referents. Would need to check if action has additional parameters and then ask for clarification or additional input



## (3B) What concept is being told about the central referent (if STATEMENT)

In [7]:
"""
In the below sentence, describe the concept that is being associated with the central referent 
sentence: The lemons on the table are sour. 
central referent: lemons 
concept: sourness
→ What do they want me to know about the thing they are talking about? 
"""

template_centralconcept = """
In the below sentence, describe the concept that is being associated with the central referent. Your answer should be a single word. 

sentence: \n{utterance}\n 
central referent: \n{centralref}
concept:
"""

prompt_centralconcept = PromptTemplate(
    input_variables=["utterance", "centralref"],
    template=template_centralconcept
)

chain_centralconcept = LLMChain(llm=llm, prompt=prompt_centralconcept)

## (4B) What "feasible concept" (if any) is being described (if STATEMENT)

In [8]:
"""
3b. CONCEPT GROUNDING (STATEMENT)
3b-i. Generate 
Given a set of known concepts… select one that is close in meaning to "concept". If several then return AMBIGUOUS. If none, then create a new concept. 
3b-ii. Test (not apply if new concept) 
Check if the parameter types of the concept sufficiently capture the nature of the referent 
→ Do I already know the thing they want me to know? 

note: if 3a or 3b requires more arguments than referents (e.g., ditransitive verb) then need to revisit the sentence 
"""

# GENERATE
template_candidateconcept = """
Select a concept from the list available concepts (or properties) that is most relevant to the given central concept as associated with the central reference and understood in the context of the utterance.
To decide the applicable concept, use the following procedure to systematically filter the most relevant concept:
1. Compare the name and description of the concept in the list of the concepts to the central concept. Narrow the list of properties to include only those with a semantically similar name or description to the central concept. 
2. If the narrowed list contains one concept, then return its name. If the narrowed list contains no concepts, then generate a new symbol with the suffix "_GENSYM" (for example a central concept "ownership" could be "own_GENSYM")

\n\n LIST OF AVAILABLE PROPERTIES/CONCEPTS \n:
{concepts}

utterance: \n{utterance}\n
central referent: \n{centralref}\n
central concept: \n{centralconcept}\n
candidate concept:

"""

prompt_candidateconcept = PromptTemplate(
    input_variables=["utterance", "concepts", "centralref", "centralconcept"],
    template=template_candidateconcept
)

chain_candidateconcept = LLMChain(llm=llm, prompt=prompt_candidateconcept)


# TEST
### Same as test available earlier. See "chain_typeof"



## (5) Generate Variables for the CPC

# Get Actions and Concepts Data

In [9]:
import re

# Helper method 
def reformat_term(input_string):
    # Extract function name
    name_match = re.match(r"(\w+)\(", input_string)
    if not name_match:
        return None

    function_name = name_match.group(1)

    # Extract parameters and type
    params_type_match = re.search(r"\((.*?)\)", input_string)
    if not params_type_match:
        return None

    params_type_string = params_type_match.group(1)
    params_type_list = params_type_string.split(',')

    parameters = []
    types = []

    for param_type in params_type_list:
        param_type = param_type.strip()

        if ':' in param_type:
            param, type_ = param_type.split(':')
            parameters.append(param.strip())
            types.append(type_.strip())
        else:
            parameters.append(param_type)
            types.append(None)

    result = {
        'name': function_name,
        'parameters': parameters,
        'types': types
    }

    return result


# Robot capabilities
actions_file = "../data/actions.txt"
concepts_file = "../data/concepts.txt"

with open(actions_file, "r") as f:
    raw_actions = f.readlines()[0]
    
with open(concepts_file, "r") as f:
    raw_concepts = f.readlines()[0]
    
actions = [reformat_term(x) for x in raw_actions.split(" ")]

concepts = [reformat_term(x) for x in raw_concepts.split(" ")]

In [10]:
concepts

[{'name': 'doit', 'parameters': ['X'], 'types': ['dialog']},
 {'name': 'dothis', 'parameters': ['Xdialog'], 'types': ['dialog']},
 {'name': 'dothat', 'parameters': ['Xdialog'], 'types': ['dialog']},
 {'name': 'that', 'parameters': ['Xdialog'], 'types': ['dialog']},
 {'name': 'this', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'any', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'physobj', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'person', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'grasp_point', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'on', 'parameters': ['X', 'Y'], 'types': ['physobj', 'physobj']},
 {'name': 'caddy', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'screwbin', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'partOf', 'parameters': ['X', 'Y'], 'types': ['physobj', 'physobj']},
 {'name': 'painkiller', 'parameters': ['X'], 'types': ['physobj']},
 {'name': 'antiseptic', 'parameters': ['X'], 'types': ['phys

## RUN PIPELINE

In [11]:
import gradio as gr
import json


# Helper Method
# Given a list of dictionaries, and a key, return the entry in the list that matches
def find_dict_in_list(lst, key, target):
    for item in lst:
        if not key in item:
            #print("Key not in Dict")
            return None
        if item[key] == target:
            return item
    #print("Nothing found")
    return None




def parse(speaker, listener, utterance, actions, concepts):
    
    output = {}
    
    # 1 Speech act classification
    speech_act = chain_speech_act_classifier.run(utterance=utterance)
    
    # 2. Central Referent Extraction
    centralref = chain_centralref.run(utterance=utterance)
    
    if speech_act == 'INSTRUCT':
        # 3A. Central Action
        centralcpc = chain_centralaction.run(utterance=utterance, centralref=centralref)
        
        done = False
        cpc_search_tries = 0
        while not done: 
            cpc_search_tries += 1
            # 4A -- Feasible Canidate Action (Generate)
            candidatecpc_name = chain_candidateaction.run(utterance=utterance,
                                                       actions=[x['name'] for x in actions],
                                                       centralref=centralref,
                                                       centralaction=centralcpc) 
            candidatecpc = find_dict_in_list(actions, 'name', candidatecpc_name)
            
            # 4A -- Test feasible candidate (Test)
            parameters = candidatecpc['parameters']
            typeof = chain_typeof.run(centralref=centralref, types=parameters).split(" ")[-1]
            typeof = ''.join(e for e in typeof if e.isalnum())
            index_of_param = -1
            if not typeof.lower() == 'none':
                index_of_param = parameters.index(typeof)
                
            output = {
                'speaker': speaker,
                'listener': listener,
                'utterance': utterance,
                'speech_act': speech_act,
                'centralref': centralref,
                'centralcpc': centralcpc,
                'candidatecpc': candidatecpc,
                'typeof': typeof,
                'index_of_param': index_of_param,
                'meta': {'cpc_search_tries': action_search_tries}

            }
            
            # TESTING HACK -- REMOVE WHEN DONE --> should be thresholding how many times Gen-test will be retried. 
            done = True
            
    elif speech_act == 'STATEMENT':
        # 3B Central Concept
        centralcpc = chain_centralconcept.run(utterance=utterance, centralref=centralref)
        
        done = False
        cpc_search_tries = 0
        while not done: 
            cpc_search_tries += 1
            # 4A -- Feasible Canidate Concept (Generate)
            candidatecpc_name = chain_candidateconcept.run(utterance=utterance,
                                                       concepts=[x['name'] for x in concepts],
                                                       centralref=centralref,
                                                       centralconcept=centralcpc) 
            
            candidatecpc = find_dict_in_list(concepts, 'name', candidatecpc_name)
            # dealing with novel concepts
            if not candidatecpc:
                candidatecpc = candidatecpc_name
                typeof = ""
                index_of_param = -1
            else:
                # 4A -- Test feasible candidate (Test)
                types = candidatecpc['types']
                typeof = chain_typeof.run(centralref=centralref, types=types).split(" ")[-1]
                typeof = ''.join(e for e in typeof if e.isalnum())
                index_of_param = -1
                if not typeof.lower() == 'none':
                    index_of_param = types.index(typeof)

            output = {
                'speaker': speaker,
                'listener': listener,
                'utterance': utterance,
                'speech_act': speech_act,
                'centralref': centralref,
                'centralcpc': centralcpc,
                'candidatecpc': candidatecpc,
                'typeof': typeof,
                'index_of_param': index_of_param,
                'meta': {'cpc_search_tries': cpc_search_tries}

            }
            
            # TESTING HACK -- REMOVE WHEN DONE --> should be thresholding how many times Gen-test will be retried. 
            done = True
        
        
    else: 
        pass
    
    
    
    return "", output


In [12]:
# Data
examples = [
    "pick up the box",
    "pick up the box that is on the table next to the lamp",
    "that thing over there is a mug",
    "that lemon belongs to Evan"
]


utterance = "that spot is where the caddy goes"
parsed, output = parse("vasanth", "self", utterance, actions, concepts)

print(parsed)
print(json.dumps(output, indent=2))


{
  "speaker": "vasanth",
  "listener": "self",
  "utterance": "that spot is where the caddy goes",
  "speech_act": "STATEMENT",
  "centralref": "Caddy",
  "centralcpc": "Location",
  "candidatecpc": {
    "name": "caddylocation",
    "parameters": [
      "VAR0"
    ],
    "types": [
      "movebaselocation"
    ]
  },
  "typeof": "NONE",
  "index_of_param": -1,
  "meta": {
    "cpc_search_tries": 1
  }
}


In [25]:


def nlu(utterance):
    parsed, output = parse("brad", "self", utterance, actions, concepts)
    return parsed, output, json.dumps(actions, indent=2), json.dumps(concepts, indent=2)

demo = gr.Interface(fn=nlu, 
                    inputs="text", 
                    outputs=["text", "text", "text", "text"],
                   examples=examples)

demo.launch(debug=True) 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
